In [1]:
import features, metrics
import ensemble

ytest:  (1778,)
Predict:  [ 773200. 1490200. 1122000. ...  345500. 1030400.  778100.]
Score:  -0.00011428023882298355


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import unittest as ut 
import sklearn.ensemble
import sklearn.metrics

class Object(object): pass
class TC(ut.TestCase): pass
tc = TC()

In [3]:
def compare_models(sk_model, our_model, X_train, X_test, y_train, y_test):
    sk_model.fit(X_train, y_train)
    our_model.fit(X_train, y_train)
    
    expected = metrics.rsq(sk_model.predict(X_train), y_train)
    actual = metrics.rsq(our_model.predict(X_train), y_train)
    print("Train, sklearn, ours:", expected, actual)
    tc.assertLess(abs(expected - actual), 0.19)
    
    expected = metrics.rsq(sk_model.predict(X_test), y_test)
    actual = metrics.rsq(our_model.predict(X_test), y_test)
    print("Test, sklearn, ours:", expected, actual)
    tc.assertLess(abs(expected - actual), 0.19)

In [4]:
income_df = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Income2.csv")
income_df = income_df.drop("Unnamed: 0", axis=1)
_income = features.Object()
_income.X = income_df.drop("Income", axis=1)
_income.y = income_df["Income"].values

In [5]:
import sklearn.model_selection
RAND_STATE = 17

_income.X_train, _income.X_test, _income.y_train, _income.y_test = \
    sklearn.model_selection.train_test_split(
        _income.X, 
        _income.y, 
        test_size=0.2, 
        random_state=RAND_STATE)

In [6]:
compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=1, random_state=RAND_STATE),
    ensemble.RandomForestRegressor(n_estimators=1, sample_ratio=1.0, random_state=RAND_STATE),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9364717923402179 0.9568283046245148
Test, sklearn, ours: 0.721713329027837 0.6963370198498757


In [7]:
compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=10, random_state=RAND_STATE),
    ensemble.RandomForestRegressor(n_estimators=10, sample_ratio=1.0, random_state=RAND_STATE),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9855590794076453 0.9870400407371869
Test, sklearn, ours: 0.8274958775158834 0.8257772237043085


In [8]:
compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=100, random_state=RAND_STATE),
    ensemble.RandomForestRegressor(n_estimators=100, sample_ratio=1.0, random_state=RAND_STATE),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9890140619420476 0.9853369363045167
Test, sklearn, ours: 0.883233461265201 0.8516038423812654


In [9]:
csv_df = pd.read_csv("Melbourne_housing_FULL.csv")

In [10]:
from sklearn.model_selection import train_test_split

_mel = Object()
_mel.X, _mel.y = features.preprocess_ver_1(csv_df, "Price")
_mel.X_train, _mel.X_test, _mel.y_train, _mel.y_test = \
    train_test_split(_mel.X, _mel.y, test_size=0.2, random_state=RAND_STATE)

In [11]:
print(_mel.X.shape, _mel.y.shape)

assert _mel.X.shape[0] == 8887, "X.shape"
assert _mel.y.shape[0] == 8887, "y.shape"

(8887, 20) (8887,)


In [12]:
_mel_sampled = Object()
_mel_sampled.X_train = _mel.X[:200]
_mel_sampled.y_train = _mel.y[:200]
_mel_sampled.X_test = _mel.X[:50]
_mel_sampled.y_test = _mel.y[:50]

In [13]:
%%time

compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=1, random_state=RAND_STATE),
    ensemble.RandomForestRegressor(n_estimators=1, sample_ratio=1.0, random_state=RAND_STATE),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

Train, sklearn, ours: 0.920353425859184 0.8552264118948804
Test, sklearn, ours: 0.8391223876039476 0.8058474438677964
CPU times: user 11.4 ms, sys: 1.75 ms, total: 13.1 ms
Wall time: 11.4 ms


In [14]:
%%time

compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=10, random_state=RAND_STATE),
    ensemble.RandomForestRegressor(n_estimators=10, sample_ratio=1.0, random_state=RAND_STATE),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

Train, sklearn, ours: 0.9606790945695954 0.9562080920036785
Test, sklearn, ours: 0.9266398707671138 0.921070263542212
CPU times: user 54 ms, sys: 2.08 ms, total: 56.1 ms
Wall time: 55.9 ms


In [15]:
%%time

compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=50, random_state=RAND_STATE),
    ensemble.RandomForestRegressor(n_estimators=50, sample_ratio=1.0, random_state=RAND_STATE),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

Train, sklearn, ours: 0.9679346890752202 0.965129692473973
Test, sklearn, ours: 0.9244637519594157 0.9124876177034095
CPU times: user 218 ms, sys: 3.59 ms, total: 221 ms
Wall time: 225 ms


In [16]:
%%time

compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=100, random_state=RAND_STATE),
    ensemble.RandomForestRegressor(n_estimators=100, sample_ratio=0.1, random_state=RAND_STATE),
    _mel.X_train, _mel.X_test, _mel.y_train, _mel.y_test)

Train, sklearn, ours: 0.9747897074093999 0.8171003661003435
Test, sklearn, ours: 0.8362731769319623 0.7943800569547488
CPU times: user 5.79 s, sys: 76.1 ms, total: 5.87 s
Wall time: 5.92 s
